In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import pathlib as pl

In [32]:
pums_folder = pl.Path('pums/csv_pus')
pums_files = [
    'psam_pusa.csv',
    'psam_pusb.csv',
    'psam_pusc.csv',
    'psam_pusd.csv'
]


In [56]:
df = pd.DataFrame()
for f in pums_files:
    df = pd.concat([
        df,
        pd.read_csv(
            pums_folder/f,
            sep=',',
            usecols=[
                'PWGTP',
                'AGEP',
                'ST', 
                'MIG',
                'MIGPUMA', # Migration PUMA based on 2010 Census definition
                'MIGSP', # Migration recode - State or foreign country code
                # 'MV'
            ],
            dtype={
                'PWGTP': np.int16,
                'AGEP': np.int16,
                'ST': np.float32, 
                'MIG': np.float32,
                'MIGPUMA': np.float32,
                'MIGSP': np.float32,
                # 'MV': np.float32,
            }
        )
    ])

df.head()

,ST,PWGTP,AGEP,MIG,MIGPUMA,MIGSP
0,1.0,40,73,1.0,NaN,NaN
1,1.0,7,31,1.0,NaN,NaN
2,1.0,46,41,1.0,NaN,NaN
3,1.0,23,48,1.0,NaN,NaN
4,1.0,21,16,1.0,NaN,NaN


In [57]:
len(df.index)

15537785

In [58]:
df = df[df.AGEP>=25]
len(df.index)

11116066

In [59]:
df[['MIG', 'MIGPUMA', 'MIGSP']].value_counts()

MIG  MIGPUMA  MIGSP
3.0  3700.0   6.0      33194
     3400.0   17.0     18386
     390.0    25.0     17512
     100.0    4.0      16998
     190.0    8.0      16833
                       ...  
2.0  1.0      414.0      233
3.0  6400.0   48.0       211
     900.0    54.0       211
2.0  1.0      427.0      191
              416.0      171
Name: count, Length: 1038, dtype: int64

In [60]:
# MIG = 3 means move within the US
df[df.MIG==3][['MIGPUMA', 'MIGSP']].value_counts()

MIGPUMA  MIGSP
3700.0   6.0      33194
3400.0   17.0     18386
390.0    25.0     17512
100.0    4.0      16998
190.0    8.0      16833
                  ...  
6900.0   48.0       259
1600.0   22.0       253
1900.0   22.0       250
900.0    54.0       211
6400.0   48.0       211
Name: count, Length: 983, dtype: int64

In [61]:
len(df[df.MIGSP==6].index)

138922

In [67]:
df[df.MIGSP==6][['ST', 'PWGTP']].groupby('ST').sum().reset_index().rename(columns={'ST': 'state_moved_from', 'PWGTP': 'num_movers'})

,state_moved_from,num_movers
0,1.0,1811
1,2.0,1799
2,4.0,41795
3,5.0,3667
4,6.0,2503307
5,8.0,18120
6,9.0,2862
7,10.0,170
8,11.0,2083
9,12.0,22074


In [69]:
states = {
    1:     'Alabama',
    2:     'Alaska',
    4:     'Arizona',
    5:     'Arkansas',
    6:     'California',
    8:     'Colorado',
    9:     'Connecticut',
    10:     'Delaware',
    11:     'District of Columbia',
    12:     'Florida',
    13:     'Georgia',
    15:     'Hawaii',
    16:     'Idaho',
    17:     'Illinois',
    18:     'Indiana',
    19:     'Iowa',
    20:     'Kansas',
    21:     'Kentucky',
    22:     'Louisiana',
    23:     'Maine',
    24:     'Maryland',
    25:     'Massachusetts',
    26:     'Michigan',
    27:     'Minnesota',
    28:     'Mississippi',
    29:     'Missouri',
    30:     'Montana',
    31:     'Nebraska',
    32:     'Nevada',
    33:     'New Hampshire',
    34:     'New Jersey',
    35:     'New Mexico',
    36:     'New York',
    37:     'North Carolina',
    38:     'North Dakota',
    39:     'Ohio',
    40:     'Oklahoma',
    41:     'Oregon',
    42:     'Pennsylvania',
    44:     'Rhode Island',
    45:     'South Carolina',
    46:     'South Dakota',
    47:     'Tennessee',
    48:     'Texas',
    49:     'Utah',
    50:     'Vermont',
    51:     'Virginia',
    53:     'Washington',
    54:     'West Virginia',
    55:     'Wisconsin',
    56:     'Wyoming',
    72:     'Puerto Rico'
}

In [79]:
df.ST = df.ST.replace(states)
df.MIGSP = df.MIGSP.replace(states)

In [72]:
df.ST.value_counts()

ST
California              1296491
Texas                    859966
Florida                  715220
New York                 692260
Pennsylvania             471249
Illinois                 445550
Ohio                     414420
Michigan                 361896
North Carolina           350099
Georgia                  327384
New Jersey               310042
Virginia                 290753
Washington               267571
Massachusetts            243191
Arizona                  238894
Tennessee                231987
Indiana                  228560
Missouri                 218741
Wisconsin                216834
Maryland                 209573
Minnesota                200810
Colorado                 196531
South Carolina           171816
Alabama                  168855
Kentucky                 156934
Louisiana                148862
Oregon                   148789
Oklahoma                 128976
Connecticut              126227
Iowa                     114621
Arkansas                 102595
Kansa

In [77]:
df[df.ST=='California'].PWGTP.sum()
# 26,801,106

26801106

In [84]:
cal_movers = df[(df.ST=='California') & (df.MIG==3) & (df.MIGSP != 'California')][['MIGSP', 'PWGTP']] \
    .groupby('MIGSP').sum().reset_index().rename(columns={'MIGSP': 'state_moved_from', 'PWGTP': 'num_movers'})
cal_movers


,state_moved_from,num_movers
0,Alabama,2193
1,Alaska,1534
2,Arizona,17318
3,Arkansas,1799
4,Colorado,11966
5,Connecticut,3144
6,Delaware,554
7,District of Columbia,3604
8,Florida,15281
9,Georgia,7233


In [94]:
total_movers = cal_movers.num_movers.sum()
cal_movers.loc[:, 'pct'] = cal_movers.num_movers / total_movers
cal_movers.sort_values('pct', ascending=False)

,state_moved_from,num_movers,pct
31,New York,26712,0.090050
43,Texas,22262,0.075048
47,Washington,20952,0.070632
2,Arizona,17318,0.058382
8,Florida,15281,0.051514
27,Nevada,14276,0.048126
12,Illinois,13894,0.046839
4,Colorado,11966,0.040339
36,Oregon,11705,0.039459
46,Virginia,11390,0.038397


In [88]:
ariz_movers = df[(df.ST=='Arizona') & (df.MIG==3) & (df.MIGSP != 'Arizona')][['MIGSP', 'PWGTP']] \
    .groupby('MIGSP').sum().reset_index().rename(columns={'MIGSP': 'state_moved_from', 'PWGTP': 'num_movers'})
ariz_movers

,state_moved_from,num_movers
0,Alabama,826
1,Alaska,1045
2,Arkansas,648
3,California,41795
4,Colorado,8196
5,Connecticut,974
6,Delaware,372
7,District of Columbia,146
8,Florida,5314
9,Georgia,2214


In [93]:
total_ariz_movers = ariz_movers.num_movers.sum()
ariz_movers.loc[:, 'pct'] = ariz_movers.num_movers / total_movers
ariz_movers.sort_values('pct', ascending=False)

,state_moved_from,num_movers,pct
3,California,41795,0.140897
47,Washington,13397,0.045163
43,Texas,9169,0.030910
4,Colorado,8196,0.027630
12,Illinois,7607,0.025644
36,Oregon,6764,0.022802
22,Minnesota,6671,0.022489
8,Florida,5314,0.017914
27,Nevada,4756,0.016033
31,New York,4476,0.015089


In [ ]:
# Things to explore:
    # net migration by state; biggest gains/losses as pct of population
    # num_movers / state moved from pop. (to account for size of state)
    # states with high pct from another state
    # income of movers
    
# What are the questions we want to answer?
    # Who the hell is moving to Phoenix?
    # What income brackets move the most?
    # What age groups move the most?

In [104]:
df = df.rename(columns={
    'ST': 'state',
    'PWGTP': 'weight',
    'AGEP': 'age',
    'MIGSP': 'state_moved_from'
})
df.columns

Index(['state', 'weight', 'age', 'MIG', 'MIGPUMA', 'state_moved_from'], dtype='object')

,state,population
0,Alabama,3415436
1,Alaska,484465
2,Arizona,4793164
3,Arkansas,2021420
4,California,26801106
5,Colorado,3938057
6,Connecticut,2516568
7,Delaware,690672
8,District of Columbia,487846
9,Florida,15350539


In [119]:
state_df = df[['state', 'weight']].groupby('state').sum().reset_index().rename(columns={'weight': 'population'})
interstate_movers = df[(df.MIG==3) & (df.state!=df.state_moved_from)]
transplants = interstate_movers[['state', 'weight']].groupby('state').sum().reset_index().rename(columns={'weight': 'num_transplants'})
leavers = interstate_movers[['state_moved_from', 'weight']] \
    .groupby('state_moved_from').sum().reset_index().rename(columns={'state_moved_from': 'state', 'weight': 'num_leavers'})
state_df = state_df.merge(transplants, how='outer').merge(leavers, how='outer')
state_df.loc[:, 'net_migration'] = state_df.num_transplants - state_df.num_leavers
state_df.sort_values('net_migration', ascending=False)

,state,population,num_transplants,num_leavers,net_migration
9,Florida,15350539.0,446425.0,295537,150888.0
43,Texas,18625270.0,338779.0,265481,73298.0
2,Arizona,4793164.0,176002.0,106675,69327.0
33,North Carolina,7086673.0,187659.0,142781,44878.0
42,Tennessee,4707471.0,129886.0,92785,37101.0
40,South Carolina,3507232.0,105797.0,76400,29397.0
28,Nevada,2115930.0,90147.0,62465,27682.0
10,Georgia,7075259.0,168557.0,143180,25377.0
12,Idaho,1185356.0,49736.0,29275,20461.0
37,Oregon,2972096.0,89134.0,73596,15538.0


In [122]:
state_df.loc[:, 'mig/pop'] = state_df.net_migration / state_df.population
state_df.sort_values('mig/pop', ascending=False)

,state,population,num_transplants,num_leavers,net_migration,mig/pop
12,Idaho,1185356.0,49736.0,29275,20461.0,0.017261
2,Arizona,4793164.0,176002.0,106675,69327.0,0.014464
28,Nevada,2115930.0,90147.0,62465,27682.0,0.013083
9,Florida,15350539.0,446425.0,295537,150888.0,0.009829
40,South Carolina,3507232.0,105797.0,76400,29397.0,0.008382
19,Maine,995057.0,23879.0,15870,8009.0,0.008049
42,Tennessee,4707471.0,129886.0,92785,37101.0,0.007881
29,New Hampshire,986133.0,33498.0,27127,6371.0,0.006461
33,North Carolina,7086673.0,187659.0,142781,44878.0,0.006333
26,Montana,745538.0,25148.0,20721,4427.0,0.005938
